
Created on Wed Mar 24 16:18 2020 (Author: Clara Burgard)

This is a script to cut out the potential temperature and practical salinity in given domains near the ice-shelf front.

It:
- calculates the distance to the ice front for the small domain in front of the ice shelf
- takes the ocean points at a given distance of the ice front and averages over them

Note that these computations can be memory-heavy! I introduced several places where you can restart the computation if it crashes in the middle. These are the places where you find "xr.open_dataset" or "xr.open_mfdataset".

REMEMBER to check if you really have potential temperature and practical salinity or if you need to convert them!!!!

In [ ]:
import xarray as xr
import numpy as np
from tqdm.notebook import tqdm
#from tqdm import tqdm
import multimelt.useful_functions as uf
import multimelt.T_S_profile_functions as tspf

from scipy.spatial import cKDTree

import distributed
import glob

READ IN THE DATA

In [ ]:
inputpath_data = # path to folder containing the file with the variable "bathy_metry"
inputpath_profiles = # path to folder containing the profiles
inputpath_isf = # path to folder containing the masks

# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

PREPARE MASK AROUND FRONT (TO RUN WITHOUT DASK!)

In [ ]:
# Read in mask file
file_isf_orig = xr.open_dataset(inputpath_isf+'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')
# Remove very small ice shelves not represented in you resolution
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
# Remove quite small ice shelves (depending on your effective resolution), we choose to remove everything below an area of 2500km2
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)

In [ ]:
# Read longitude and latitude
lon = file_isf['longitude']
lat = file_isf['latitude']

In [ ]:
# Read in bathymetry to define continental shelf
file_mask_orig = # path to file containing variable "bathy_metry"
file_mask = uf.cut_domain_stereo(file_mask_orig, map_lim, map_lim).squeeze().drop('time')

In [ ]:
# Read in just one yearly file to compute spatial characteristics
T_S_ocean_oneyear = xr.open_dataset(inputpath_profiles+'T_S_theta_ocean_corrected_2000.nc')

In [ ]:
# Define ocean points not covered by ice shelves
ocean = np.isfinite(T_S_ocean_oneyear['theta_ocean'].isel(time=0,depth=0)).drop('time').drop('depth')
# only points below 1500 m
offshore = file_mask['bathy_metry'] > 1500 # .drop('lon').drop('lat')
# only points above 1500 m
contshelf = file_mask['bathy_metry'] <= 1500 # .drop('lon').drop('lat')

In [ ]:
#mask_domains = (ocean & contshelf).load() #<= checked if it does what it should and it does! :)
#mask_domains = (ocean).load()

# define one domain on the continental shelf and one domain "offshore"
mask_domains = xr.DataArray([(ocean & contshelf), (ocean & offshore)],
                            dims={'profile_domain': ['close_cont_shelf','offshore'], 'y': contshelf.y, 'x': contshelf.x}).load()

# define lon_box and lat_box where to compute the distance to the ice-shelf front
lon_box = xr.DataArray(np.array([10.0, 10.0]), coords=[('profile_domain', ['close_cont_shelf','offshore'])])
lat_box = xr.DataArray(np.array([3.5, 3.5]), coords=[('profile_domain', ['close_cont_shelf','offshore'])])  

In [ ]:
# Prepare masks around the ice shelf front over which we want to take the mean profiles.
close_region_around_isf_mask = tspf.mask_boxes_around_IF_new(lon, lat, mask_domains, 
                                    file_isf['front_min_lon'], file_isf['front_max_lon'], 
                                    file_isf['front_min_lat'], file_isf['front_max_lat'],  
                                    lon_box, lat_box, 
                                    file_isf['isf_name'])

In [ ]:
# Compute distance of ocean points to the ice-shelf front in the masked region on the continental shelf (for each ice shelf) and write to netcdf
dist_list = [ ]
for kisf in tqdm(file_isf['Nisf']):
        
        if (file_isf['IF_mask']==kisf).sum() > 0:
            region_to_cut_out = close_region_around_isf_mask.sel(profile_domain='close_cont_shelf').sel(Nisf=kisf)
            region_to_cut_out = region_to_cut_out.where(region_to_cut_out > 0, drop=True)
            IF_region = file_isf['IF_mask'].where(file_isf['IF_mask']==kisf, drop=True)

            dist_from_front = tspf.distance_isf_points_from_line_small_domain(region_to_cut_out,IF_region)
            dist_list.append(dist_from_front)
    
dist_all = xr.concat(dist_list, dim='Nisf').reindex_like(file_isf)
dist_all.to_dataset(name='dist_from_front').to_netcdf(inputpath_profiles+'dist_to_ice_front_only_contshelf_oneFRIS.nc')


In [ ]:
# the offshore mask is ready already and can be written to netcdf directly - no need for distance from the grounding line, it is just a box offshore of the ice shelf
close_region_around_isf_mask.sel(profile_domain='offshore').to_dataset(name='mask').to_netcdf(inputpath_profiles+'mask_offshore_oneFRIS.nc')

COMPUTING THE MEAN PROFILES

DEPENDING ON THE SIZE OF YOUR DATA, THIS WILL REQUIRE USING DASK

For example, open a client like this: (check that the number of workers is lower or equal to the number of cores you use and the memory limit is equal to the memory of your cores)

From experience it makes sense to restart your kernel and restart here to erase memory used by previous operations

In [ ]:
client = distributed.Client(n_workers=16, dashboard_address=':8795', local_directory='/tmp', memory_limit='4GB')

In [ ]:
client

CONTINENTAL SHELF

In [ ]:
# define the different domains on the continental shelf
bbox_da = xr.DataArray(np.array([10000., 25000., 50000., 100000.]), coords=[('dist_from_front', [10,25,50,100])])

If workers don't die: with 12 cores, took approx 1hour. If workers die, divide work by years (all_in_one = False)

In [ ]:
all_in_one = False # False if worker die if you put in all years at once, True if workers don't die 
# about chunking: the values here are the most "efficient" ones for me here, might require adjusting if your workers have more/less memory available
if all_in_one:
    dist_to_front_file = xr.open_mfdataset(inputpath_profiles+'dist_to_ice_front_only_contshelf_oneFRIS.nc',chunks={'x': 50, 'y': 50})
    T_S_ocean_files = xr.open_mfdataset(inputpath_profiles+'T_S_theta_ocean_corrected_*.nc', concat_dim='time', chunks={'x': 50, 'y': 50, 'depth': 50}, parallel=True)
    T_S_ocean_oneyear = xr.open_mfdataset(inputpath_profiles+'T_S_theta_ocean_corrected_2000.nc',chunks={'x': 50, 'y': 50, 'depth': 50})
else:
    dist_to_front_file = xr.open_mfdataset(inputpath_profiles+'dist_to_ice_front_only_contshelf_oneFRIS.nc',chunks={'x': 100, 'y': 100})
    T_S_ocean_files = xr.open_mfdataset(inputpath_profiles+'T_S_theta_ocean_corrected_*.nc', concat_dim='time', chunks={'x': 100, 'y': 100, 'depth': 50}, parallel=True) 
    T_S_ocean_oneyear = xr.open_mfdataset(inputpath_profiles+'T_S_theta_ocean_corrected_2000.nc',chunks={'x': 100, 'y': 100, 'depth': 50}) #
dist_to_front = dist_to_front_file['dist_from_front']

MAKING THE MEAN DIRECTLY NEEDS TO MUCH MEMORY 

so we divide the steps to make the mean: (1) the sum of T and S over the domain of interest, (2) the sum of the points of interest, (3) dividing (1) by (2)

Prepare sum (1)

In [ ]:
# mask of the distance domain we want
mask_km = dist_to_front <= bbox_da

In [ ]:
# sum over all T and S in that region
ds_sum = (T_S_ocean_files * mask_km).sum(['x','y'])

In [ ]:
# check format
ds_sum

In [ ]:
# write this sum to netcdf (this will start the stuff in dask)
if all_in_one:
    ds_sum = ds_sum.load()
    ds_sum.to_netcdf(inputpath_profiles+'ds_sum_for_mean_contshelf.nc')
else:
    yearly_datasets = list(tspf.split_by_chunks(ds_sum.unify_chunks(),'time'))
    paths = [tspf.create_filepath(ds, 'ds_sum_for_mean_contshelf', inputpath_profiles, ds.time[0].values) for ds in yearly_datasets]
    xr.save_mfdataset(datasets=yearly_datasets, paths=paths)

Prepare number of points by which you divide: sum (2)

In [ ]:
if all_in_one:
    ds_sum = xr.open_mfdataset(inputpath_profiles+'ds_sum_for_mean_contshelf.nc')
else:
    ds_sum = xr.open_mfdataset(inputpath_profiles+'ds_sum_for_mean_contshelf*.nc', concat_dim='time', parallel=True).drop('profile_domain')

In [ ]:
# mask points in domain of interest in depth where there is water
mask_depth = T_S_ocean_oneyear['salinity_ocean'].squeeze().drop('time') > 0
mask_all = mask_km & mask_depth

In [ ]:
# sum all points in domain of interest in depth where there is water, keep depth dimension
mask_sum = mask_all.sum(['x','y'])

In [ ]:
# start the tasks in dask
mask_sum = mask_sum.load()

Make the mean (3) - divide (1) by (2)

In [ ]:
ds_mean = ds_sum/mask_sum

In [ ]:
ds_mean = ds_mean.drop('profile_domain').rename({'dist_from_front': 'profile_domain'})

In [ ]:
# write to netcdf
ds_mean.to_netcdf(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_1980-2018.nc')

OFFSHORE PROFILES

Same procedure as above

In [ ]:
T_S_ocean_files = xr.open_mfdataset(inputpath_profiles+'T_S_theta_ocean_corrected_*.nc', concat_dim='time', chunks={'x': 1000, 'y': 1000, 'depth': 50}, parallel=True)
T_S_ocean_oneyear = xr.open_mfdataset(inputpath_profiles+'T_S_theta_ocean_corrected_2000.nc',chunks={'x': 1000, 'y': 1000, 'depth': 50})

In [ ]:
mask_offshore_file = xr.open_mfdataset(inputpath_profiles+'mask_offshore_oneFRIS.nc',chunks={'x': 1000, 'y': 1000}).sel(Nisf=[11])
mask_offshore = mask_offshore_file['mask'].drop('profile_domain')
mask_depth = T_S_ocean_oneyear['salinity_ocean'].squeeze().drop('time') > 0
mask_all_offshore = mask_offshore & mask_depth

Sum (1)

In [ ]:
ds_sum_offshore = (T_S_ocean_files * mask_offshore).sum(['x','y'])
ds_sum_offshore['profile_domain'] = np.array([1000])

In [ ]:
ds_sum_offshore = ds_sum_offshore.load()
ds_sum_offshore.to_netcdf(inputpath_profiles+'ds_sum_for_mean_offshore.nc')

Sum (2)

In [ ]:
mask_sum_offshore = mask_all_offshore.sum(['x','y'])

In [ ]:
mask_sum_offshore = mask_sum_offshore.load()

(3) Divide (1) by (2)

In [ ]:
ds_mean_offshore = ds_sum_offshore/mask_sum_offshore

In [ ]:
ds_mean_offshore.to_netcdf(inputpath_profiles+'T_S_mean_prof_corrected_km_offshore_1980-2018.nc')

COMBINE BOTH

In [ ]:
ds_mean_offshore = xr.open_dataset(inputpath_profiles+'T_S_mean_prof_corrected_km_offshore_1980-2018.nc')
ds_mean = xr.open_dataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_1980-2018.nc')

In [ ]:
ds_mean_both = xr.concat([ds_mean, ds_mean_offshore], dim='profile_domain')
ds_mean_both.to_netcdf(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_and_offshore_1980-2018.nc')